This notebook takes an input sql file, parametrizes relevant queries and outputs an sql file containing a larger, parametrized set of queries

In [2]:
import re
import math
from datetime import datetime, date

Import sql queries

In [3]:
with open('tpch.txt') as file: #Todo- change this to a sql file..
    all_queries = file.read()

In [4]:
query_list = all_queries.split(';')
print(len(query_list))
print(query_list[5])

24


-- TPC-H 6
select
	sum(l_extendedprice * l_discount) as revenue
from
	snowflake_sample_data.tpch_sf1.lineitem
where
	l_shipdate >= date '1994-01-01'
	AND l_shipdate < DATEADD(year, 1, '1994-01-01')
	and l_discount between .06 - 0.01 and .06 + 0.01
	and l_quantity < 24


Parametrizing by dateadd functions

In [5]:
# final_query_output = []
# for query in query_list:
#     #Split query by dateadd function into a list of query 'fragments'.
#     arglist = re.split("dateadd", query, flags=re.IGNORECASE)
    
#     #if there is at least one dateadd in the query...
#     if len(arglist) > 1:
#         for query_fragment in arglist[1:]:
#             int_to_change = query_fragment.split(',')[1].strip() #first element will be the numeric argument of dateadd
            
#             outer_index = query.index(query_fragment)
#             inner_index = query_fragment.index(int_to_change)
#             full_index = outer_index + inner_index
            
#             #update the parameter and save off to a new query for each mod factor.
#             for mod in [0.5, 1, 2]:
#                 modded_int = math.ceil(mod * int(int_to_change))
#                 modded_query = query[:full_index] + int_to_change + query[full_index+len(int_to_change):]
#                 final_query_output.append(modded_query)
                
#     #no dateadd, so just add query
#     else:
#         final_query_output.append(query)
    
        
#     arglist = re.split(" date ", query, flags=re.IGNORECASE)
    
#     if len(arglist) > 1:
#         for query_fragment in arglist[1:]:
#             datetochange = query_fragment[1:11]
#             initialDate = datetime.strptime(datetochange, '%Y-%m-%d')

#             outer_index = query.index(query_fragment)
#             inner_index = query_fragment.index(datetochange)
#             full_index = outer_index + inner_index

#             for mod in [0, 2, 4]:
#                 newDate = date(initialDate.year + (mod*2), initialDate.month, initialDate.day)
#                 newDate = str(newDate)
#                 modded_query = query[:full_index] + newDate + query[full_index+len(newDate):]
#                 final_query_output.append(modded_query)
    

In [10]:
final_query_output = []
for query in query_list:
    #Split query by dateadd function into a list of query 'fragments'.
    arglist = re.split("dateadd", query, flags=re.IGNORECASE)
    
    #if there is at least one dateadd in the query...
    if len(arglist) > 1:
        for query_fragment in arglist[1:]:
            int_to_change = query_fragment.split(',')[1].strip() #first element will be the numeric argument of dateadd
            date_to_change = query_fragment.split(',')[2][0:12].strip() #second element will be the year
            print(int_to_change)
            print(date_to_change)
            
            outer_index = query.index(query_fragment)
            inner_index = query_fragment.index(int_to_change)
            full_index_int = outer_index + inner_index
            
            outer_index = query.index(query_fragment)
            inner_index = query_fragment.index(date_to_change)
            full_index_date = outer_index + inner_index
            
            #update with $varname: https://stackoverflow.com/questions/35217048/how-can-i-pass-variable-into-a-sql-file-while-running-through-python
            modded_query = query[:full_index_int] + '$quantity' + ', ' + '$date' + query[full_index_date+len(date_to_change) + 1:]
            final_query_output.append(modded_query)
                
    #no dateadd, so just add query
    else:
        final_query_output.append(query)
    

90
'1998-12-01
3
'1993-07-01
1
'1994-01-01
1
'1994-01-01
3
'1993-10-01
1
'1994-01-01
1
'1995-09-01
3
'1996-01-01
1
'1994-01-01


In [11]:
print(len(final_query_output))

24


In [12]:
sql_string = ";\n".join([q for q in final_query_output])
print(sql_string)

-- TPC-H 1
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= DATEADD(day, $quantity, $date)
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus;


-- TPC-H 2
select
	s_acctbal,
	s_name,
	n_name,
	p_partkey,
	p_mfgr,
	s_address,
	s_phone,
	s_comment
from
	part,
	supplier,
	partsupp,
	nation,
	region
where
	p_partkey = ps_partkey
	and s_suppkey = ps_suppkey
	and p_size = 15
	and p_type like '%BRASS'
	and s_nationkey = n_nationkey
	and n_regionkey = r_regionkey
	and r_name = 'EUROPE'
	and ps_supplycost = (
		select
			min(ps_supplycost)
		from
			partsupp,
			supplier,
			nation,
			region
		where
			p_partkey = ps_partkey
			an

In [25]:
# Write to output SQL file... 
with open('output.sql', 'w') as outfile:
    outfile.write(sql_string)

Todo:
- Get rid of overlap in dateadds to prevent caching.
- Find other arguments to parametrize programatically